<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Onehot_A" data-toc-modified-id="Onehot_A-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Onehot_A</a></span></li><li><span><a href="#导入与分割数据" data-toc-modified-id="导入与分割数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入与分割数据</a></span></li><li><span><a href="#A_cat" data-toc-modified-id="A_cat-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A_cat</a></span></li><li><span><a href="#A_hour" data-toc-modified-id="A_hour-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A_hour</a></span></li><li><span><a href="#A_xgb" data-toc-modified-id="A_xgb-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>A_xgb</a></span></li><li><span><a href="#B_cat" data-toc-modified-id="B_cat-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>B_cat</a></span></li><li><span><a href="#A_his" data-toc-modified-id="A_his-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>A_his</a></span></li><li><span><a href="#结论" data-toc-modified-id="结论-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>结论</a></span></li><li><span><a href="#'C':-0.1" data-toc-modified-id="'C':-0.1-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>'C': 0.1</a></span></li></ul></div>

# Onehot_A

In [53]:

import numpy as np
import pandas as pd
import time
import gc
import os
import scipy.sparse as ss
#from  sklearn.cross_validation  import  train_test_split 
import xgboost as xgb
#from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import GridSearchCV


from scipy.spatial.distance import jaccard, cosine

from sklearn.externals.joblib import dump, load

import utils




In [54]:
# 数据路径
dpath = utils.dpath
# 数据类型
data_types = utils.data_types
# 缓存数据路径
tmp_dpath = utils.tmp_dpath
# 距离计算公式
get_distance = utils.get_distance
to_0_1 = utils.normalization
to_cat = utils.label_encoder

# 导入users和events的index索引, 以及相关信息
users_index = load(tmp_dpath+'users_index.joblib.gz')
events_index = load(tmp_dpath+'events_index.joblib.gz')
all_user = set(users_index.keys())
all_event = set(events_index.keys())
num_users = len(users_index)
num_events = len(events_index)
user_event = load(tmp_dpath+'user_event.joblib.gz')
user_event_scores = load(tmp_dpath+'user_event_scores.joblib.gz')
data_df = load(tmp_dpath+'data_df.joblib.gz')

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f78f0115668>>
Traceback (most recent call last):
  File "/home/zhou/anaconda3/lib/python3.5/site-packages/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


# 导入与分割数据

In [55]:
train = data_df.iloc[:15398,:].copy()
test = data_df.iloc[15398:,:].copy()

In [56]:
train[:5]

,event,interested,invited,not_interested,user,user_id,event_id,day,month,weekday,date,user_cf_dis,event_cf_dis,user_cf_reco,event_cf_reco,events_yes_num,events_all_num,users_freds_num
0,1918771225,0.0,0,0.0,3044012,1982,3396,1.033333,1.75,1.166667,1.689956,1.650327,1.684758,1.649747,1.589014,1.0008,1.0058,1.175499
1,1502284248,0.0,0,0.0,3044012,1982,1067,1.033333,1.75,1.166667,1.689956,1.666667,1.702021,1.666667,1.583333,1.0010,1.0140,1.175499
2,2529072432,1.0,0,0.0,3044012,1982,2302,1.033333,1.75,1.166667,1.689956,1.787475,1.823584,1.833625,1.000000,1.0212,1.4347,1.175499
3,3072478280,0.0,0,0.0,3044012,1982,7976,1.033333,1.75,1.166667,1.689956,1.666667,1.719680,1.666667,1.583333,1.0008,1.0023,1.175499
4,1390707377,0.0,0,0.0,3044012,1982,294,1.033333,1.75,1.166667,1.689956,1.692661,1.697196,1.696304,1.673430,1.0083,1.4017,1.175499


In [57]:
#导入数据
print('Load Data')
y_train = train['interested']
user_event = train[['event', 'user',]].copy()
X_train = train.drop(['event', 'user', 'interested', 'not_interested','user_id','event_id'],axis=1)
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [58]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15398 entries, 0 to 15397
Data columns (total 12 columns):
invited            15398 non-null int8
day                15398 non-null float64
month              15398 non-null float64
weekday            15398 non-null float64
date               15398 non-null float64
user_cf_dis        15398 non-null float64
event_cf_dis       15398 non-null float64
user_cf_reco       15398 non-null float64
event_cf_reco      15398 non-null float64
events_yes_num     15398 non-null float64
events_all_num     15398 non-null float64
users_freds_num    15398 non-null float64
dtypes: float64(11), int8(1)
memory usage: 1.4 MB


In [59]:
X_train.shape, y_train.shape, 

((15398, 12), (15398,))

In [60]:
xgtrain = xgb.DMatrix(X_train, label = y_train,)


In [61]:
#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=9, 
        min_child_weight=5,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)


num_trees = 16 #树的数量##################

#调用cv函数
bst_train = xgb.cv(param, xgtrain, num_trees, nfold=5, stratified=True)
#bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))
bst_train

training . . . 
cost time:0


,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.208751,0.002953,0.204919,0.000412
1,0.102295,0.004599,0.096362,0.002038
2,0.061622,0.003641,0.054188,0.002473
3,0.042075,0.001381,0.033549,0.002265
4,0.032004,0.002610,0.022827,0.001017
5,0.025594,0.002376,0.016378,0.001088
6,0.022147,0.002368,0.013157,0.000674
7,0.020225,0.002860,0.011226,0.000315
8,0.018888,0.002810,0.009785,0.000463
9,0.017896,0.003403,0.008862,0.000273


In [62]:
xgb_train = xgb.train(param, xgtrain, num_trees, ) 

In [63]:
X_test = test.drop(['event', 'user', 'interested', 'not_interested','user_id','event_id'],axis=1)
xgtestn = xgb.DMatrix(X_test, )

X_test[:5]

,invited,day,month,weekday,date,user_cf_dis,event_cf_dis,user_cf_reco,event_cf_reco,events_yes_num,events_all_num,users_freds_num
15398,0,1.966667,1.875,1.666667,1.947598,1.0,1.0,1.0,1.0,1.0061,1.0107,1.089686
15399,0,1.966667,1.875,1.666667,1.947598,1.0,1.0,1.0,1.0,1.0018,1.0104,1.089686
15400,0,1.966667,1.875,1.666667,1.947598,1.0,1.0,1.0,1.0,1.0026,1.0085,1.089686
15401,0,1.966667,1.875,1.666667,1.947598,1.0,1.0,1.0,1.0,1.0031,1.0100,1.089686
15402,0,1.966667,1.875,1.666667,1.947598,1.0,1.0,1.0,1.0,1.0021,1.0076,1.089686


In [64]:
X_train.max()

invited            1.000000e+00
day                2.000000e+00
month              2.000000e+00
weekday            2.000000e+00
date               2.000000e+00
user_cf_dis        2.165430e+00
event_cf_dis       1.999917e+00
user_cf_reco       2.170313e+00
event_cf_reco      1.050840e+16
events_yes_num     2.000000e+00
events_all_num     2.000000e+00
users_freds_num    2.000000e+00
dtype: float64

In [65]:
X_test.max()

invited            1.000000
day                2.000000
month              2.000000
weekday            2.000000
date               2.000000
user_cf_dis        1.529412
event_cf_dis       1.000000
user_cf_reco       1.000000
event_cf_reco      1.000000
events_yes_num     1.980300
events_all_num     2.000000
users_freds_num    1.988993
dtype: float64

In [66]:
train_preds = xgb_train.predict(xgtrain)

In [67]:
test_preds = xgb_train.predict(xgtestn)


In [68]:
train_preds.max(), test_preds.max()

(0.99998331, 0.025548982)

In [69]:
user_event = test.loc[:,['user','event']].copy()
user_event.index = test['user']
user_event['pred'] = test_preds
user_event[:5]

,user,event,pred
user,,,
1776192,1776192,2877501688,0.000272
1776192,1776192,3025444328,0.000272
1776192,1776192,4078218285,0.000272
1776192,1776192,1024025121,0.000272
1776192,1776192,2972428928,0.000272


In [70]:
submission = pd.Series(index = set(user_event.index),)
for u in set(user_event.index):
    data_tmp = user_event.loc[u,:]
    event_tmp = pd.Series(data_tmp['pred'].values, index = data_tmp['event'].values)
    events = event_tmp.sort_values(ascending=False).index.values
    submission[u] = ' '.join(events.astype(np.str))

In [71]:
submission

2686965763    940261225 2103350077 1335898508 3546861867 270...
629450757     955398943 2529072432 2093319113 3655295959 260...
511678471     955398943 1138354613 1600413013 1557387709 157...
3730114567    2973911742 1628013431 569253671 268816201 1302...
2678956041    834360181 3118576636 2235916835 4130552113 921...
4180064266    2102453106 2180806657 2658555390 2658555390 71...
4095465484    172445691 569616021 814858005 2130383503 17489...
2872854546    955398943 457253413 2980258101 2392886869 7906...
3270633490    1269035551 2007279414 4244463632 3309338938 67...
666374166     786960145 2407779614 899115709 1944996876 2052...
1753157654    252715505 1536770720 4073937824 2974348707 410...
3329238167    401945277 3429610508 3323634350 1882674933 313...
1645299738    2007279414 2073609284 3950589515 2149464820 12...
3264469018    4074266568 4258255746 2350143315 389608207 169...
2371436573    2529072432 861118590 1716825317 656598905 7947...
1551265824           3981414613 99711967

In [76]:
submission.to_csv(tmp_dpath+'submission.csv')

In [28]:
for c,v in zip(X_train.columns, lr.coef_[0], ):
    print(c,':',v)

invited 	 0.556743988549
day 	 -0.626827364025
month 	 -3.12328376375
weekday 	 0.827367955752
date 	 2.10007720179
user_cf_dis 	 -41.750450971
event_cf_dis 	 22.2571095547
user_cf_reco 	 68.520086545
events_yes_num 	 -1.29234156032
events_all_num 	 -1.09697376686
users_freds_num 	 -0.389572682185
